<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/selectivity_neuron_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, hidden_layer_neurons):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, hidden_layer_neurons)
        self.linear_2 = torch.nn.Linear(hidden_layer_neurons, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 30
def selectivity_trainer(optimizer, model, hidden_layer_neurons):

    hidden_layer_each_neuron = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(hidden_layer_neurons)]
    hidden_layer_each_neuron = np.array(hidden_layer_each_neuron)

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

            # Print/Append activation of the hidden layer 
            # print(model.layer_activations.shape)
            # model.layer_activations

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # find selectivity at the final epoch 
            if epoch == no_epochs - 1: # last epoch 
                for activation, label in zip(model.layer_activations, labels):
                    # shape of activation and label: 256 and 1 
                    
                    # get the actual value of item. This is because label is now Tensor 
                    label = label.item()

                    # this is not part of gradient calculcation 
                    with torch.no_grad():
                        activation = activation.numpy()

                    # for each image/label, append activation value of neuron 
                    for i in range(hidden_layer_neurons):    # number of neurons in hidden layer 
                        hidden_layer_each_neuron[i][label].append(activation[i])


        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
        #     print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
        #     torch.save(model.state_dict(), "model.dth")

    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(hidden_layer_neurons)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(hidden_layer_neurons)]

    # selectivity_list contains all of the selectivity of each neuron 
    selectivity_list = list()

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity_list
        selectivity_list.append(selectivity)

    return test_acc, selectivity_list

In [ ]:
hidden_layer_neurons = [64, 128, 256, 384, 512, 768, 1024]

# AdaDelta

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adadelta = Model(hidden_layer_neurons[i])
    print("model_Adadelta:", model_Adadelta)
    model_Adadelta.to(device)
    model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
    optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
    Adadelta_test_acc, Adadelta_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_selectivity_Adadelta.txt", "w")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')
    else:
        f = open("neurons_selectivity_Adadelta.txt", "a")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')

f.close()

!cp neurons_selectivity_Adadelta.txt /content/drive/MyDrive

Entering 0-th loop
model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.45272081, Test Loss: 0.23829513, Test Accuracy: 0.93360000

Epoch: 2/30, Train Loss: 0.20915445, Test Loss: 0.17909380, Test Accuracy: 0.94690000

Epoch: 3/30, Train Loss: 0.16421468, Test Loss: 0.14749705, Test Accuracy: 0.95850000

Epoch: 4/30, Train Loss: 0.13715678, Test Loss: 0.13100217, Test Accuracy: 0.96050000

Epoch: 5/30, Train Loss: 0.11852796, Test Loss: 0.11825826, Test Accuracy: 0.96600000

Epoch: 6/30, Train Loss: 0.10498591, Test Loss: 0.11073469, Test Accuracy: 0.96720000

Epoch: 7/30, Train Loss: 0.09372254, Test Loss: 0.10677438, Test Accuracy: 0.96720000

Epoch: 8/30, Train Loss: 0.08516297, Test Loss: 0.10170035, Test Accuracy: 0.97040000

Epoch: 9/30, Train Loss: 0.07819226, Test Loss: 0.09489400, Test Accuracy: 0.97190000

Epoch: 10/30, Trai

# AdaGrad

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adagrad = Model(hidden_layer_neurons[i])
    print("model_Adagrad:", model_Adagrad)
    model_Adagrad.to(device)
    model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
    optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
    Adagrad_test_acc, Adagrad_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_selectivity_Adagrad.txt", "w")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')
    else:
        f = open("neurons_selectivity_Adagrad.txt", "a")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')

f.close()

!cp neurons_selectivity_Adagrad.txt /content/drive/MyDrive

Entering 0-th loop
model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.25470983, Test Loss: 0.15037494, Test Accuracy: 0.95530000

Epoch: 2/30, Train Loss: 0.12868607, Test Loss: 0.12286012, Test Accuracy: 0.96360000

Epoch: 3/30, Train Loss: 0.10050087, Test Loss: 0.11054668, Test Accuracy: 0.96550000

Epoch: 4/30, Train Loss: 0.08345022, Test Loss: 0.10360545, Test Accuracy: 0.96800000

Epoch: 5/30, Train Loss: 0.07192038, Test Loss: 0.09705938, Test Accuracy: 0.96910000

Epoch: 6/30, Train Loss: 0.06383208, Test Loss: 0.09383416, Test Accuracy: 0.97020000

Epoch: 7/30, Train Loss: 0.05750150, Test Loss: 0.09226996, Test Accuracy: 0.97130000

Epoch: 8/30, Train Loss: 0.05210986, Test Loss: 0.08874171, Test Accuracy: 0.97140000

Epoch: 9/30, Train Loss: 0.04764512, Test Loss: 0.08874024, Test Accuracy: 0.97150000

Epoch: 10/30, Train

# SGD


In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_SGD = Model(hidden_layer_neurons[i])
    print("model_SGD:", model_SGD)
    model_SGD.to(device)
    model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
    optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
    SGD_test_acc, SGD_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_selectivity_SGD.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')
    else:
        f = open("neurons_selectivity_SGD.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')

f.close()

!cp neurons_selectivity_SGD.txt /content/drive/MyDrive

Entering 0-th loop
model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.84010790, Test Loss: 0.38418883, Test Accuracy: 0.89820000

Epoch: 2/30, Train Loss: 0.34567705, Test Loss: 0.29593623, Test Accuracy: 0.91760000

Epoch: 3/30, Train Loss: 0.29020889, Test Loss: 0.26605766, Test Accuracy: 0.92460000

Epoch: 4/30, Train Loss: 0.25994722, Test Loss: 0.24099451, Test Accuracy: 0.93120000

Epoch: 5/30, Train Loss: 0.23785265, Test Loss: 0.22121951, Test Accuracy: 0.93870000

Epoch: 6/30, Train Loss: 0.21944983, Test Loss: 0.20620261, Test Accuracy: 0.94170000

Epoch: 7/30, Train Loss: 0.20428450, Test Loss: 0.19488495, Test Accuracy: 0.94470000

Epoch: 8/30, Train Loss: 0.19112998, Test Loss: 0.18264221, Test Accuracy: 0.94770000

Epoch: 9/30, Train Loss: 0.18001741, Test Loss: 0.17621129, Test Accuracy: 0.94720000

Epoch: 10/30, Train Los

# Adam

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adam = Model(hidden_layer_neurons[i])
    print("model_Adam:", model_Adam)
    model_Adam.to(device)
    model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
    optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
    Adam_test_acc, Adam_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_selectivity_Adam.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')
    else:
        f = open("neurons_selectivity_Adam.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')

f.close()

!cp neurons_selectivity_Adam.txt /content/drive/MyDrive

Entering 0-th loop
model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.58806727, Test Loss: 0.28062321, Test Accuracy: 0.92200000

Epoch: 2/30, Train Loss: 0.24281302, Test Loss: 0.20646097, Test Accuracy: 0.94020000

Epoch: 3/30, Train Loss: 0.19095139, Test Loss: 0.17552695, Test Accuracy: 0.94940000

Epoch: 4/30, Train Loss: 0.15998729, Test Loss: 0.15357532, Test Accuracy: 0.95510000

Epoch: 5/30, Train Loss: 0.13784190, Test Loss: 0.13857591, Test Accuracy: 0.96050000

Epoch: 6/30, Train Loss: 0.12040462, Test Loss: 0.12792508, Test Accuracy: 0.96360000

Epoch: 7/30, Train Loss: 0.10663814, Test Loss: 0.11991986, Test Accuracy: 0.96500000

Epoch: 8/30, Train Loss: 0.09543831, Test Loss: 0.11120653, Test Accuracy: 0.96740000

Epoch: 9/30, Train Loss: 0.08585681, Test Loss: 0.10547914, Test Accuracy: 0.97040000

Epoch: 10/30, Train Lo